In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import ishermitian
import sys
import os
import pyzx as zx


# Import ZXW functions

from pauli_hamiltonian_zx import PauliHamiltonianZX


from cor_decay_zxw import (
    compute_F_jk_equation5,
    create_equation4_hamiltonian,
    setup_positions_3d_grid,
    setup_positions_2d_grid
)

## Setup: Atom Positions and F_jk Matrix


### Parameters


In [2]:
N = 4 # Number of atoms (small for visualization)
lambda_val = 1.0  # Wavelength
gam = 1.0  # Decay rate Γ
m = 4  # 1d-size of array relative to lambda

### Setting up 2D Grid Positions

In [3]:
two_dim_positions = setup_positions_2d_grid(N, m, lambda_val)
print(two_dim_positions)
for i in range(len(two_dim_positions[0])):
    print(f"  Atom {i}: ({two_dim_positions[0][i]:.3f}, {two_dim_positions[1][i]:.3f})")

(array([0., 0., 2., 2.]), array([0., 2., 0., 2.]), array([0., 0., 0., 0.]))
  Atom 0: (0.000, 0.000)
  Atom 1: (0.000, 2.000)
  Atom 2: (2.000, 0.000)
  Atom 3: (2.000, 2.000)


# Compute $F_{jk}$ matrix using equation 5

In [4]:

F_matrix = compute_F_jk_equation5(*two_dim_positions, lambda_val, gam)

print(f"F_jk matrix shape: {F_matrix.shape}")
print(f"\nF_jk matrix (real part):")
print(F_matrix)

print(f"\nSample coupling values:")
for j in range(min(3, N)):
    for k in range(j+1, min(3, N)):
        print(f"  F_{j}{k} = {F_matrix[j, k].real:.6f} + {F_matrix[j, k].imag:.6f}i")


F_jk matrix shape: (4, 4)

F_jk matrix (real part):
[[0.+0.j         0.-0.00474943j 0.-0.00474943j 0.+0.03594016j]
 [0.-0.00474943j 0.+0.j         0.+0.03594016j 0.-0.00474943j]
 [0.-0.00474943j 0.+0.03594016j 0.+0.j         0.-0.00474943j]
 [0.+0.03594016j 0.-0.00474943j 0.-0.00474943j 0.+0.j        ]]

Sample coupling values:
  F_01 = 0.000000 + -0.004749i
  F_02 = 0.000000 + -0.004749i
  F_12 = 0.000000 + 0.035940i


## Creating Pauli string Hamiltonian for equation 4
### It uses: 
$H_jk = (F_{jk}/2) * (X_j X_k + Y_j Y_k),$   $j \neq k$

$H_jj = -i (\Gamma/2) * (Z_j + I_j)$

In [5]:
pauli_strings = create_equation4_hamiltonian(N, F_matrix, gamma=gam)

# UNCOMMENT to see all Pauli strings

In [6]:
# print(f"Number of Pauli terms: {len(pauli_strings)}")
# print(f"\nFirst 6 Pauli strings (showing X_j X_k and -Y_j Y_k pairs):")

# for i, (coeff, gates) in enumerate(pauli_strings[:]):
#     print(f"  {i+1}: {coeff:.6f} * {gates}")


# Create ZXW Hamiltonian
Using list of Pauli Strings

In [7]:
hamiltonian = PauliHamiltonianZX(pauli_strings)
print(f"\nTotal qubits: {hamiltonian.total_qubits}")


Total qubits: 4


# Build the ZXW diagram

In [8]:
print("Building ZXW diagram...")
graph = hamiltonian.build_graph()
print("Graph built successfully")

Building ZXW diagram...
Graph built successfully


# Vizualize our Graph

In [9]:
print("\nDisplaying ZXW diagram:")
zx.draw(graph)


Displaying ZXW diagram:


# Choosing Trotter Parameters

In [10]:
vals =  [[]]
max_steps = 2
t_f = 3
t_vals = np.linspace(0.0, t_f, 30)

In [11]:
trotter_graph = hamiltonian.build_single_trotter_symbolic(steps=max_steps)
# Export to json for visualization
file_name = "example_graphs//symbolic_graph.json"
zx.draw(trotter_graph)
# Save to file
json = trotter_graph.to_json()
with open(file_name, "w") as f:
    f.write(json)
print(f"Symbolic trotter graph saved to {file_name}")

########## Function [PauliHamiltonianZX.build_single_trotter_symbolic()] is not implemented correctly yet. ##########
Node 11:
  - Type: 6 (1=Z, 2=X, 3=H_BOX)
  - Phase (from .phase()): -0.0007558953382781085j
Node 19:
  - Type: 6 (1=Z, 2=X, 3=H_BOX)
  - Phase (from .phase()): -0.0007558953382781085j
Node 27:
  - Type: 6 (1=Z, 2=X, 3=H_BOX)
  - Phase (from .phase()): -0.0007558953382781085j


TypeError: Poly.__init__() takes 2 positional arguments but 3 were given

### In **pyZX** as it stands the best way to symbolically represent these trotter expansion graphs is to hide the complex valued nature of the weights within the variable, as it stands I am leaving it as is with print statements in the symbolic functions to note the incomplete nature of it.